<a href="https://colab.research.google.com/github/ihasdapie/roadSignClassification/blob/master/TrafficSigns_PreProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# grab datasets
!wget -q https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip
!wget -q https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip
!wget -q https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip
#unpack data
!unzip -qq /content/GTSRB_Final_Test_GT.zip
!unzip -qq /content/GTSRB_Final_Test_Images.zip
!unzip -qq /content/GTSRB_Final_Training_Images.zip


In [ ]:
!mkdir cookies


In [ ]:
!cp  /content/drive/My\ Drive/GTSRB/Processed_GTSRB_Dataset.zip /content/Processed_GTSRB.zip

In [ ]:
!unzip GTSRB_Processed.zip -d sandbox

Streaming output truncated to the last 5000 lines.
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00064_00024.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00054_00024.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00055_00029.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00035_00028.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00041_00005.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00014_00000.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00043_00024.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00029_00008.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00044_00002.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00044_00027.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00019_00003.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Images/00038/00015_00022.ppm  
  inflating: sandbox/P_GTSRB/Final_Training/Image

In [ ]:
mkdir sandbox

In [ ]:
# imports
import glob
import numpy as np
from skimage import color, exposure, transform, io
import os
import cv2


In [ ]:
# Constants
NUM_CLASSES = 43
IMAGE_SIZE=48  

In [ ]:
def processImage(rgbImg):
    hsvImg = color.rgb2hsv(rgbImg)
     # hsv-equalize brightness only
    hsvImg[:, :, 2] = exposure.equalize_adapthist(hsvImg[:, :, 2])
    rgbImg=color.hsv2rgb(hsvImg)

    # turn into max-size-ed centered squares.
    if (rgbImg.shape[0] != rgbImg.shape[1]):
        smallestSide=min(rgbImg.shape[:-1])
        center = ((rgbImg.shape[0]//2),(rgbImg.shape[1]//2))
        rgbImg = rgbImg[(center[0]-(smallestSide//2)):(center[0]+smallestSide),
                          (center[1]-(smallestSide//2)):(center[1]+smallestSide),:]
    rgbImg = transform.resize(rgbImg, (IMAGE_SIZE, IMAGE_SIZE))
    return rgbImg

In [ ]:
# let's pre-process the images
from multiprocessing.dummy import Pool

@timeit
def processAllImages(paths):
    pool = Pool(20)
    img_res = {}
    # for i in range(len(paths)):
    for i in range(8000): #processing all the images takes a really long
        img_res[i] = pool.apply_async(processImage, args=(cv2.imread(paths[i]), ))
    pool.close()
    for i in range(max(img_res.keys())):
        img_res[i] = img_res[i].get()
    out = [img_res[i] for i in range(max(img_res.keys()))]
    return out

images = processAllImages(PATHS)

In [ ]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [ ]:
BASE_DIR = '/content/GTSRB/Final_Test/Images/'
PATHS = [BASE_DIR+x for x in os.listdir(BASE_DIR) if "ppm" in x]
print(PATHS)

['/content/GTSRB/Final_Test/Images/10984.ppm', '/content/GTSRB/Final_Test/Images/01779.ppm', '/content/GTSRB/Final_Test/Images/07410.ppm', '/content/GTSRB/Final_Test/Images/06731.ppm', '/content/GTSRB/Final_Test/Images/01513.ppm', '/content/GTSRB/Final_Test/Images/08111.ppm', '/content/GTSRB/Final_Test/Images/02872.ppm', '/content/GTSRB/Final_Test/Images/03333.ppm', '/content/GTSRB/Final_Test/Images/06601.ppm', '/content/GTSRB/Final_Test/Images/05540.ppm', '/content/GTSRB/Final_Test/Images/11702.ppm', '/content/GTSRB/Final_Test/Images/05855.ppm', '/content/GTSRB/Final_Test/Images/08499.ppm', '/content/GTSRB/Final_Test/Images/08315.ppm', '/content/GTSRB/Final_Test/Images/12002.ppm', '/content/GTSRB/Final_Test/Images/09764.ppm', '/content/GTSRB/Final_Test/Images/05867.ppm', '/content/GTSRB/Final_Test/Images/03780.ppm', '/content/GTSRB/Final_Test/Images/10339.ppm', '/content/GTSRB/Final_Test/Images/04202.ppm', '/content/GTSRB/Final_Test/Images/08456.ppm', '/content/GTSRB/Final_Test/Images

In [ ]:
WRITE_PATHS = [s.replace("GTSRB", "P_GTSRB", 1) for s in PATHS]
print(WRITE_PATHS)

['/content/P_GTSRB/Final_Test/Images/10984.ppm', '/content/P_GTSRB/Final_Test/Images/01779.ppm', '/content/P_GTSRB/Final_Test/Images/07410.ppm', '/content/P_GTSRB/Final_Test/Images/06731.ppm', '/content/P_GTSRB/Final_Test/Images/01513.ppm', '/content/P_GTSRB/Final_Test/Images/08111.ppm', '/content/P_GTSRB/Final_Test/Images/02872.ppm', '/content/P_GTSRB/Final_Test/Images/03333.ppm', '/content/P_GTSRB/Final_Test/Images/06601.ppm', '/content/P_GTSRB/Final_Test/Images/05540.ppm', '/content/P_GTSRB/Final_Test/Images/11702.ppm', '/content/P_GTSRB/Final_Test/Images/05855.ppm', '/content/P_GTSRB/Final_Test/Images/08499.ppm', '/content/P_GTSRB/Final_Test/Images/08315.ppm', '/content/P_GTSRB/Final_Test/Images/12002.ppm', '/content/P_GTSRB/Final_Test/Images/09764.ppm', '/content/P_GTSRB/Final_Test/Images/05867.ppm', '/content/P_GTSRB/Final_Test/Images/03780.ppm', '/content/P_GTSRB/Final_Test/Images/10339.ppm', '/content/P_GTSRB/Final_Test/Images/04202.ppm', '/content/P_GTSRB/Final_Test/Images/084

In [ ]:
s = "!mkdir P_GTSRB/Final_Training/Images/"
for x in range(43):
    sx = str(x)
    po = ""
    for x in range(5-len(sx)):
        po=po+"0"
    print(s+po+sx)

In [ ]:
!mkdir P_GTSRB/Final_Test/Images

In [ ]:
!mkdir P_GTSRB/Final_Training/Images/00000
!mkdir P_GTSRB/Final_Training/Images/00001
!mkdir P_GTSRB/Final_Training/Images/00002
!mkdir P_GTSRB/Final_Training/Images/00003
!mkdir P_GTSRB/Final_Training/Images/00004
!mkdir P_GTSRB/Final_Training/Images/00005
!mkdir P_GTSRB/Final_Training/Images/00006
!mkdir P_GTSRB/Final_Training/Images/00007
!mkdir P_GTSRB/Final_Training/Images/00008
!mkdir P_GTSRB/Final_Training/Images/00009
!mkdir P_GTSRB/Final_Training/Images/00010
!mkdir P_GTSRB/Final_Training/Images/00011
!mkdir P_GTSRB/Final_Training/Images/00012
!mkdir P_GTSRB/Final_Training/Images/00013
!mkdir P_GTSRB/Final_Training/Images/00014
!mkdir P_GTSRB/Final_Training/Images/00015
!mkdir P_GTSRB/Final_Training/Images/00016
!mkdir P_GTSRB/Final_Training/Images/00017
!mkdir P_GTSRB/Final_Training/Images/00018
!mkdir P_GTSRB/Final_Training/Images/00019
!mkdir P_GTSRB/Final_Training/Images/00020
!mkdir P_GTSRB/Final_Training/Images/00021
!mkdir P_GTSRB/Final_Training/Images/00022
!mkdir P_GTSRB/Final_Training/Images/00023
!mkdir P_GTSRB/Final_Training/Images/00024
!mkdir P_GTSRB/Final_Training/Images/00025
!mkdir P_GTSRB/Final_Training/Images/00026
!mkdir P_GTSRB/Final_Training/Images/00027
!mkdir P_GTSRB/Final_Training/Images/00028
!mkdir P_GTSRB/Final_Training/Images/00029
!mkdir P_GTSRB/Final_Training/Images/00030
!mkdir P_GTSRB/Final_Training/Images/00031
!mkdir P_GTSRB/Final_Training/Images/00032
!mkdir P_GTSRB/Final_Training/Images/00033
!mkdir P_GTSRB/Final_Training/Images/00034
!mkdir P_GTSRB/Final_Training/Images/00035
!mkdir P_GTSRB/Final_Training/Images/00036
!mkdir P_GTSRB/Final_Training/Images/00037
!mkdir P_GTSRB/Final_Training/Images/00038
!mkdir P_GTSRB/Final_Training/Images/00039
!mkdir P_GTSRB/Final_Training/Images/00040
!mkdir P_GTSRB/Final_Training/Images/00041
!mkdir P_GTSRB/Final_Training/Images/00042

In [ ]:
print(cv2.imread(PATHS[0]).shape)
print(processImage(cv2.imread(PATHS[0])).shape)

In [ ]:
# let's pre-process the images
for i in range(len(PATHS)):
    print(PATHS[i].split("/")[-1], WRITE_PATHS[i].split("/")[-1])
    cv2.imwrite(WRITE_PATHS[i], processImage(cv2.imread(PATHS[i])))

Streaming output truncated to the last 5000 lines.
09782.ppm 09782.ppm
12629.ppm 12629.ppm
11745.ppm 11745.ppm
03838.ppm 03838.ppm
11285.ppm 11285.ppm
02280.ppm 02280.ppm
07498.ppm 07498.ppm
05400.ppm 05400.ppm
01753.ppm 01753.ppm
00753.ppm 00753.ppm
03056.ppm 03056.ppm
10406.ppm 10406.ppm
10555.ppm 10555.ppm
02407.ppm 02407.ppm
04550.ppm 04550.ppm
11832.ppm 11832.ppm
02447.ppm 02447.ppm
06218.ppm 06218.ppm
12558.ppm 12558.ppm
09504.ppm 09504.ppm
02616.ppm 02616.ppm
04198.ppm 04198.ppm
08447.ppm 08447.ppm
06747.ppm 06747.ppm
07699.ppm 07699.ppm
09307.ppm 09307.ppm
03327.ppm 03327.ppm
00842.ppm 00842.ppm
04953.ppm 04953.ppm
04479.ppm 04479.ppm
00326.ppm 00326.ppm
02739.ppm 02739.ppm
00296.ppm 00296.ppm
06697.ppm 06697.ppm
09751.ppm 09751.ppm
04579.ppm 04579.ppm
04317.ppm 04317.ppm
01913.ppm 01913.ppm
03528.ppm 03528.ppm
01073.ppm 01073.ppm
10642.ppm 10642.ppm
03668.ppm 03668.ppm
00853.ppm 00853.ppm
10868.ppm 10868.ppm
10758.ppm 10758.ppm
12513.ppm 12513.ppm
08155.ppm 08155.ppm
10691.ppm

In [ ]:
!zip -r GTSRB_Processed.zip P_GTSRB

Streaming output truncated to the last 5000 lines.
  adding: P_GTSRB/Final_Training/Images/00038/00064_00024.ppm (deflated 95%)
  adding: P_GTSRB/Final_Training/Images/00038/00054_00024.ppm (deflated 92%)
  adding: P_GTSRB/Final_Training/Images/00038/00055_00029.ppm (deflated 98%)
  adding: P_GTSRB/Final_Training/Images/00038/00035_00028.ppm (deflated 92%)
  adding: P_GTSRB/Final_Training/Images/00038/00041_00005.ppm (deflated 94%)
  adding: P_GTSRB/Final_Training/Images/00038/00014_00000.ppm (deflated 92%)
  adding: P_GTSRB/Final_Training/Images/00038/00043_00024.ppm (deflated 96%)
  adding: P_GTSRB/Final_Training/Images/00038/00029_00008.ppm (deflated 92%)
  adding: P_GTSRB/Final_Training/Images/00038/00044_00002.ppm (deflated 94%)
  adding: P_GTSRB/Final_Training/Images/00038/00044_00027.ppm (deflated 94%)
  adding: P_GTSRB/Final_Training/Images/00038/00019_00003.ppm (deflated 95%)
  adding: P_GTSRB/Final_Training/Images/00038/00015_00022.ppm (deflated 92%)
  adding: P_GTSRB/Final_T

In [ ]:
!cp GTSRB_Processed.zip /content/drive/My\ Drive/GTSRB_Processed.zip